In [1]:
# ノンパラメトリック法とは？
# 正規分布などの母集団分布の仮定を設けない状態で仮設検定を行うもの

In [13]:
# 2郡間の差の検定1 ウィルコクソンの順位和検定

# 帰無仮説：2郡A,Bの成績の分布は同じ
# 対立仮設：郡Aの成績の分布の形は郡Bと同じだが、悪い(-)方にズレている(片側検定)

import numpy as np
from scipy.special import comb

# サンプルデータを用意し、A,Bの２郡に分ける
x = np.array([30., 20., 52., 40., 50., 35.])
x_A = x[:3]
x_B = x[3:]

# 郡Aと郡Bのデータを合わせて、小さい順に順位を与える(1~)
rank_A = np.argsort(x)[:3]+1
rank_B = np.argsort(x)[3:]+1

# 郡ごとに順位の和を計算する
W_A = np.sum(rank_A)
W_B = np.sum(rank_B)

# P値を求める。P値=郡Aの順位和/全データに与えられる順位の組合せ数
comb_all = comb(len(x), len(x_A), exact=True)
P_value = W_A/comb_all
P_value

0.45

In [15]:
# 2郡間の差の検定2 並べ替え検定

# 帰無仮説：2郡A,Bの成績の分布は同じ
# 対立仮設：郡Aの成績の分布の形は郡Bと同じだが、悪い(-)方にズレている(片側検定)

import numpy as np
import itertools
from scipy.special import comb

# サンプルデータを用意し、A,Bの２郡に分ける
x = np.array([30., 20., 52., 40., 50., 35.])
x_A = x[:3]
x_B = x[3:]

# 郡ごとの平均値を求める
avg_A = np.average(x_A)
avg_B = np.average(x_B)

# P値を求める。郡Aと同じサンプル数を抜き出した際の平均値が郡A以下になる確率
count_target = 0
count_comb_all = comb(len(x), len(x_A), exact=True)
for comb in itertools.combinations(x, 3):
    if sum(comb)/len(x_A) <= avg_A:
        count_target+=1

P_value = count_target/count_comb_all
P_value


0.25

In [41]:
# 対応がある場合の差の検定1 (ウィルコクソンの)符号付き順位検定

# 帰無仮説：分布Dの中央値が0
# 対立仮設：分布Dの中央値が0以上(片側検定)
# 分布Dの対称性を仮定し、介入前と介入後で効果があったかどうかを検定

import numpy as np

X_before = np.array([30., 20., 52., 40., 50., 35., 25.])
X_after = np.array([15., 11., 52., 46., 61., 55., 50.])

# 差分を算出する
D_val = X_after - X_before

# 値が0の観測値は除いた、符号付きの順位を算出する
D_val_nonzero = D_val[np.nonzero(D_val)]
rank_unsigned = np.argsort(abs(D_val_nonzero))
D_val_nonzero = D_val_nonzero[rank_unsigned.tolist()] 
rank_signed = np.array(range(1, len(rank_unsigned)+1))
for i in rank_signed.tolist():
    if D_val_nonzero[i-1] < 0:
        rank_signed[i-1] = rank_signed[i-1] * -1

# 正値の合計T+を検定統計量とする
T_plus_val = rank_signed[rank_signed > 0].sum()

# 全ての順位の±の組み合わせ数のうち、T_plus_val以上となる確率をP値とする。処理は割愛する


15


In [50]:
# scipyを使用して算出。正規分布への近似を利用して計算する

from scipy import stats

import numpy as np

X_before = np.array([30., 20., 52., 40., 50., 35., 25.])
X_after = np.array([15., 11., 52., 46., 61., 55., 50.])
result = stats.wilcoxon(X_before, X_after, alternative='greater')
result

/Users/yasukawakazuki/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/yasukawakazuki/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


WilcoxonResult(statistic=6.0, pvalue=0.8272762347653871)

In [57]:
# 対応がある場合の差の検定2 符号検定

# 帰無仮説：分布Dの中央値が0
# 対立仮設：分布Dの中央値が0以上(片側検定)
# D分布の対称性は仮定しない

# 帰無仮説が正しいとき、検定統計量T+は二項分布Bin(n, 0.5)に従うと考えてP値を算出する。
# nは、0の値を取らないサンプル数
# scipyを使用して算出。

from scipy import stats

import numpy as np

# 正になった個数を検定統計量T+とする
X_before = np.array([30., 20., 52., 40., 50., 35., 25.])
X_after = np.array([15., 11., 52., 46., 61., 55., 50.])
D_val = X_after - X_before
T_plus_val = np.count_nonzero(D_val > 0)
non_zero_num = np.count_nonzero(D_val != 0)

# P値を算出
result = stats.binom_test(x=T_plus_val, n=non_zero_num, p=0.5, alternative='greater')
result


0.34375

In [58]:
# 3郡以上の差の検定 クラスカル・ウォリス検定

# 帰無仮説：全ての郡の分布は同じ
# 対立仮設：全ての郡の分布は同じではない
# scipyを使用して算出。

import numpy as np
from scipy import stats

group_a = np.array([22., 30., 42.])
group_b = np.array([36., 40., 52., 53.])
group_c = np.array([25., 32., 45., 48.])

result = stats.mstats.kruskalwallis(group_a, group_b, group_c)
result

KruskalResult(statistic=2.962121212121211, pvalue=0.2273963824597735)

In [59]:
# 順位相関係数とは？
# 2次元データが共に順位データである場合の相関係数のこと

In [62]:
# スピアマンの順位相関係数

import numpy as np

x_val = np.array([1., 2., 3., 4., 5., 6., 7.])
y_val = np.array([1., 3., 2., 6., 4., 5., 7.])
diff = x_val - y_val
square_diff_val = np.square(diff).sum()
n = len(x_val)

# スピアマンの順位相関係数を計算
rs_val = 1 - (6*square_diff_val / (n*(n**(2)-1)) )
rs_val

0.8571428571428572

In [64]:
# scipyを使用して算出した場合

import scipy.stats as st

corr, pvalue = st.spearmanr(x_val, y_val)
corr # 上記と同じ値を算出

0.8571428571428573

In [70]:
# ケンドールの順位相関係数

x_val = np.array([1., 2., 3., 4., 5., 6., 7.])
y_val = np.array([1., 3., 2., 6., 4., 5., 7.])

# (xi, yi)と(xj, yj)について(xi-xj)*(yi-yj)が正となる組の数をP、負となる数をNとする
# この時、i≠jかつi<jを満たす

P_count = 0
N_count = 0
n = len(x_val)

for i in range(len(x_val)):
    for j in range(len(x_val)):
        if i >= j:
            continue

        val = (x_val[i]-x_val[j])*(y_val[i]-y_val[j])
        if val > 0:
            P_count+=1
        else:
            N_count+=1

# ケンドールの順位相関係数を計算
rk_val = (P_count - N_count) / (n*(n-1)/2)
rk_val

0.7142857142857143

In [69]:
# scipyを使用して算出した場合

import scipy.stats as st

corr, pvalue = st.kendalltau(x_val, y_val)
corr # 上記と同じ値を算出

0.7142857142857143